# 环境

In [17]:
import torch
import torchmetrics
import torchvision
import os
import gc
import torch.nn as nn
from torchvision import transforms
print(torch.__version__)
device='cuda' if torch.cuda.is_available() else 'cpu'
print(device)
if device=='cuda':
    gc.collect()
    torch.cuda.empty_cache()
print(os.getcwd())


1.11.0+cu113
cuda
/root/autodl-tmp/deep-learning/unsupervised denoise


# 模型

In [18]:
class HourGlassCNNBlock(nn.Module):
    def __init__(self,in_channels,out_channels,size=128,layernorm=False,batchnorm=False,activation=True):
        super(HourGlassCNNBlock,self).__init__()
        layers=[]
        layers.append(nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1))
        if layernorm:layers.append(nn.LayerNorm([out_channels,size,size]))
        if batchnorm:layers.append(nn.BatchNorm2d(out_channels))
        if activation:layers.append(nn.SiLU())
        self.main=nn.Sequential(*layers)
    def forward(self,x):
        return self.main(x)#channel不同不能进行残差学习
class HourGlassCNN(nn.Module):
    def __init__(self,size=128,max_channels=512,layernorm=False,batchnorm=False):
        super(HourGlassCNN,self).__init__()
        self.blocks=
class NoiseExtractorMain(nn.Module):
    def __init__(self,size=128,layernorm=False,batchnorm=False):
        super(NoiseExtractorMain,self).__init__()
        self.blocks=nn.ModuleList([
            HourGlassCNNBlock(3,64,size,layernorm=False,batchnorm=False),#0
            HourGlassCNNBlock(64,128,size,layernorm,batchnorm),#1
            HourGlassCNNBlock(128,256,size,layernorm,batchnorm),#2
            
            HourGlassCNNBlock(256,512,size,layernorm,batchnorm),#3

            HourGlassCNNBlock(512,256,size,layernorm,batchnorm),#4
            HourGlassCNNBlock(256,128,size,layernorm,batchnorm),#5
            HourGlassCNNBlock(128,64,size,layernorm,batchnorm),#6
            HourGlassCNNBlock(64,3,size,layernorm=False,batchnorm=False,activation=False),#7
        ])
    def forward(self,x):
        y0=self.blocks[0](x)
        y1=self.blocks[1](y0)
        y2=self.blocks[2](y1)
        y3=self.blocks[3](y2)

        y4=self.blocks[4](y3)+y2
        y5=self.blocks[5](y4)+y1
        y6=self.blocks[6](y5)+y0
        y7=self.blocks[7](y6)
        return y7
class Denoiser(nn.Module):
    def __init__(self,size=(128,128),layernorm=False,batchnorm=False):
        super(Denoiser,self).__init__()
        self.blocks=nn.ModuleList([
            HourGlassCNNBlock(3,64,size,layernorm=False,batchnorm=False),#0
            HourGlassCNNBlock(64,128,size,layernorm,batchnorm),#1
            HourGlassCNNBlock(128,256,size,layernorm,batchnorm),#2
            
            HourGlassCNNBlock(256,512,size,layernorm,batchnorm),#3

            HourGlassCNNBlock(512,256,size,layernorm,batchnorm),#4
            HourGlassCNNBlock(256,128,size,layernorm,batchnorm),#5
            HourGlassCNNBlock(128,64,size,layernorm,batchnorm),#6
            HourGlassCNNBlock(64,3,size,layernorm=False,batchnorm=False,activation=False),#7
        ])
    def forward(self,x):
        y0=self.blocks[0](x)
        y1=self.blocks[1](y0)
        y2=self.blocks[2](y1)
        y3=self.blocks[3](y2)

        y4=self.blocks[4](y3)+y2
        y5=self.blocks[5](y4)+y1
        y6=self.blocks[6](y5)+y0
        y7=self.blocks[7](y6)+x
        return y7

# 分析

In [19]:
import torch.utils.tensorboard
from torch.utils.tensorboard.writer import SummaryWriter
from tqdm.notebook import tqdm
from torchvision.utils import make_grid
from matplotlib import pyplot as plt
from torchmetrics import PeakSignalNoiseRatio

In [20]:
def convert_to_rgb255(image:torch.Tensor):
    image = (image + 1) / 2
    image[image < 0] = 0
    image[image > 1] = 1
    return image
def show_image(image:torch.Tensor):
    image=convert_to_rgb255(image)
    plt.imshow(transforms.ToPILImage()(image))
    plt.show()

writer=SummaryWriter()
psnr=PeakSignalNoiseRatio().to(device=device)

# 数据

In [21]:
img_size=128
batch_size=8

In [22]:

from torch.utils.data import Dataset 
from torch.utils.data import DataLoader

from PIL import Image
# https://discuss.pytorch.org/t/torchvision-transfors-how-to-perform-identical-transform-on-both-image-and-target/10606/7

class BSDSPairsDataSet(Dataset):
    def __init__(self,imgs_dir=None,img_size=128,sigma=30) -> None:
        super(BSDSPairsDataSet,self).__init__()
        if not imgs_dir:imgs_dir=f'./data/BSDS300/{img_size}x{img_size}/base/train'
        self.img_size=img_size
        self.files=os.listdir(imgs_dir)
        self.sigma=sigma
        self.imgs_dir=imgs_dir
    def __getitem__(self,idx:int):
        img_path = os.path.join(self.imgs_dir, self.files[idx])
        clean = Image.open(img_path).convert('RGB')#比torchvision的好
        transform = transforms.Compose([
            # hwc->chw 并 归一化到[0,1]
            transforms.ToTensor(),
            # [−1,1]
            transforms.Normalize((.5, .5, .5), (.5, .5, .5)),
            ])
        clean = transform(clean)
        noisy = clean + self.sigma * torch.randn(clean.shape)/255
        return noisy, clean
    def __len__(self):
        return len(self.files)

train_set=BSDSPairsDataSet()
train_loader=DataLoader(train_set,batch_size=batch_size,shuffle=True)


# 训练

In [23]:
import itertools
from torch.optim import Adam
noise_extractor = NoiseExtractor(layernorm=False,batchnorm=False).to(device)
denoiser = Denoiser(layernorm=False,batchnorm=False).to(device)
noise_extractor_optimizer = Adam(noise_extractor.parameters() , lr=1e-3)
denoiser_optimizer=Adam(denoiser.parameters() , lr=1e-3)

In [24]:
# noise as clean：noisy-P'->noisy' 如果clean-P->noisy中的P近似于P'那么(noisy',noisy)训练可以近似复原噪声图片
from torch import log, var, zeros_like,zeros,abs
from torch.nn import SmoothL1Loss
from torch.nn.functional import relu
smooth_l1_loss=SmoothL1Loss(beta=0.1)
def train(noisys):
    noise_extractor_optimizer.zero_grad()
    denoiser_optimizer.zero_grad()

    cleans=denoiser(noisys)
    noise=noise_extractor(noisys)
    
    loss1=smooth_l1_loss(noise_extractor(cleans),zeros_like(noisys))
    loss2=smooth_l1_loss(noise_extractor(noise),noise)
    loss3=smooth_l1_loss(denoiser(cleans),cleans)
    loss4=smooth_l1_loss(denoiser(noise),zeros_like(noisys))

    fake_noisys=cleans+noise
    loss5=smooth_l1_loss(fake_noisys,noisys)
    loss6=smooth_l1_loss(denoiser(fake_noisys),cleans)
    loss7=smooth_l1_loss(noise_extractor(fake_noisys),noise)

    noise_var=var(noise*255,unbiased=False,dim=0)
    noise_var_var=var(noise_var,unbiased=False)

    loss8=relu(-batch_size*log(smooth_l1_loss(noise_var,zeros(size=(3,128,128)).to(device=device))))
    loss9=batch_size*smooth_l1_loss(noise_var_var,zeros_like(noise_var_var))

    
    loss=loss1+loss2+loss3+loss4+loss5+loss6+loss7+loss8+loss9
    loss.backward()

    noise_extractor_optimizer.step()
    denoiser_optimizer.step()
    return loss.sum()
def draw(mean_loss_noisy,mean_loss_clean,epoch):
    noisys,cleans=next(iter(train_loader))
    noisys=noisys.to(device=device)
    cleans=cleans.to(device=device)
    outs=denoiser(noisys)
    writer.add_scalar('noisy_loss',mean_loss_noisy,epoch)
    writer.add_scalar('clean_loss',mean_loss_clean,epoch)
    writer.add_scalar('psnr',psnr(outs,cleans),epoch)
    writer.add_image('noisy',make_grid(convert_to_rgb255(noisys)),epoch)
    writer.add_image('clean',make_grid(convert_to_rgb255(cleans)),epoch)
    writer.add_image('out',make_grid(convert_to_rgb255(outs)),epoch)
    writer.add_image('noise',make_grid(convert_to_rgb255(noise_extractor(noisys))),epoch)
def pretrain():
    pass
def main(): 
    for epoch in tqdm(range(5000)):
        total_loss_noisy,total_loss_clean=0,0
        for noisys,cleans in train_loader:
            if noisys.shape[0]!=batch_size:break
            noisys=noisys.to(device=device)
            cleans=cleans.to(device=device)
            total_loss_noisy+=train(noisys)
            with torch.no_grad():
                total_loss_clean+=smooth_l1_loss(denoiser(noisys),cleans).sum()
        with torch.no_grad():
            draw(total_loss_noisy,total_loss_clean,epoch)
main()

  0%|          | 0/5000 [00:00<?, ?it/s]

KeyboardInterrupt: 